In [22]:
import pickle
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import svm

In [23]:
nClusters = 256
# we will get VLAD of size nClusters*128
nComponents = 1024
svm_c = 0.01
svm_gamma = 0.01
svm_kernal = 'rbf'

datasetPath = './dataset/cifar-10-batches-py'
vocabFile = 'vocab_'+str(nClusters)+'.sav'
clsFile = 'cls.sav'
trainBatch = 5
'''trainImgPerBatch = 50
testImgPerBatch = 5'''

In [24]:
def getLabel(id):
    txtLabels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    return txtLabels[id]

In [25]:
def loadBatch(filePath):
    with open(filePath, 'rb') as fo:
        batchDict = pickle.load(fo, encoding='latin1')
        nImgs = len(batchDict['data'])
        imgs = batchDict['data'].reshape(nImgs,3,32,32).transpose(0, 2, 3, 1)
        labels = batchDict['labels']
    return (imgs,labels)

In [26]:
def getDescriptors(img, extractor):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = extractor.detectAndCompute(gray, None)
    return descriptors

In [27]:
def getBatchDesc(imgs):
    descriptors = np.asarray([])
    extractor = cv2.xfeatures2d.SIFT_create()
    #i = 0
    for img in imgs:
        desc = getDescriptors(img,extractor)
        if type(desc) == np.ndarray :
            if descriptors.shape[0] == 0:
                descriptors = desc
            else:
                descriptors = np.concatenate((descriptors, desc), axis=0)
    return descriptors

In [28]:
def getDatasetDesc():
    descriptors = np.asarray([])
    for batchId in range(1,trainBatch+1):
        print('collecting batch ',batchId,'decriptors')
        batchPath = datasetPath+'/'+'data_batch_'+str(batchId)
        imgs,labels = loadBatch(batchPath)
        imgs = imgs[:trainImgPerBatch]
        if descriptors.shape[0] == 0:
            descriptors = getBatchDesc(imgs)
        else:
            descriptors = np.concatenate((descriptors, getBatchDesc(imgs)), axis=0)
    return descriptors

In [29]:
def getVocabularies():
    descriptors = getDatasetDesc()
    kmeans = KMeans(n_clusters = nClusters)
    kmeans.fit(descriptors)
    return kmeans

In [30]:
def getVLAD(img,extractor,vocab):
    descDim = 128 # 128 = dimention of descriptor
    VLAD = [ [0]*descDim ]*nClusters 
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = extractor.detectAndCompute(gray, None)
    if type(descriptors) == np.ndarray:
        pred = vocab.predict(descriptors)
        
        for i in range(descriptors.shape[0]):
            relDist = descriptors[0] - vocab.cluster_centers_[pred[0]]
            for j in range(descDim):
                VLAD[pred[0]][j] = relDist[j]
                
        flattenVLAD = [ele for li_1d in VLAD for ele in li_1d]
            return flattenVLAD
    else:
        return [ 0 ]*(descDim * nClusters)

In [31]:
def getBatchVLAD(imgs,vocab):
    batchBOK = []
    extractor = cv2.xfeatures2d.SIFT_create()
    for img in imgs:
        bok = getVLAD(img,extractor,vocab)
        batchBOK.append(bok)
    return  np.asarray(batchBOK)

In [32]:
def getTrainingDT(vocab):
    dataset = np.asarray([])
    labels = []
    for batchId in range(1,trainBatch+1):
        print('collecting batch ',str(batchId),' features ...')
        batchPath = datasetPath+'/'+'data_batch_'+str(batchId)
        imgs,labs = loadBatch(batchPath)
        #imgs = imgs[:trainImgPerBatch]             #--------------------------------------
        if dataset.shape[0] == 0:
            dataset = getBatchVLAD(imgs,vocab)
            labels = labs
        else:
            dataset = np.concatenate((dataset, getBatchVLAD(imgs,vocab)), axis=0)
            labels.extend(labs)
        #labels = labels[:trainImgPerBatch]         #--------------------------------------
    return (dataset,np.asarray(labels))

In [33]:
def getTestingDT(vocab):
    batchPath = datasetPath+'/test_batch'
    imgs,labels = loadBatch(batchPath)
    #imgs = imgs[:testImgPerBatch]                 #--------------------------------------
    #labels = labels[:testImgPerBatch]             #--------------------------------------
    dataset = getBatchVLAD(imgs,vocab)
    return (dataset,np.asarray(labels))

In [34]:
vocab = getVocabularies()
pickle.dump(vocab, open(vocabFile, 'wb'))

#vocab = pickle.load(open(vocabFile, 'rb'))

collecting batch  1 decriptors


In [35]:
dataset,labels = getTrainingDT(vocab)

collecting batch  1  features ...


In [36]:
from sklearn.decomposition import PCA
pca = PCA(n_components=nComponents)
priComTrain = pca.fit_transform(dataset)

In [37]:
print('saving VLAD ...')
saveData = np.concatenate((priComTrain,np.asarray([labels]).T),axis=1)
np.savetxt('vlad.csv',saveData, delimiter=',',fmt='%8.4f')

'''loadData = np.loadtxt('vlad.csv',delimiter=',')
labels = loadData[:,-1]
priComTrain = loadData[:,:-1]'''

saving VLAD ...


"loadData = np.loadtxt('vlad.csv',delimiter=',')\nlabels = loadData[:,-1]\npriComTrain = loadData[:,:-1]"

In [38]:
#model = svm.SVC(C=10,kernel='rbf',gamma=0.01)
model = svm.SVC(C=svm_c,kernel=svm_kernal,gamma=svm_gamma)
#model = svm.SVC(C=svm_c,kernel=svm_kernal)
print("training started...")
model.fit(priComTrain,labels)
pickle.dump(model, open(clsFile, 'wb'))

training started...


In [39]:
testDt,testLb = getTestingDT(vocab)
priComTest = pca.fit_transform(testDt)

In [40]:
#out = model.predict(testDt)
acc = model.score(priComTest,testLb)

In [41]:
print(acc)

0.2
